<a href="https://colab.research.google.com/github/Romkaxomka/Laba_ABD/blob/main/4Laba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Часть 1.
#Выполнить парсинг вакансий любого рекрутингового сайта (HeadHunter, Indeed, SuperJob, Зарплата, и тд) с фильтром по отрасли ИТ.

import requests
import json
import time
import os
import csv
 
  
def getPage(page = 0):
  params = {
      'area': 95, # Поиск ощуществляется по вакансиям города Москва
      'page': page, # Индекс страницы поиска на HH
      'per_page': 100 # Кол-во вакансий на 1 странице
  }

  req = requests.get('https://api.hh.ru/vacancies', params)
  data = req.content.decode()
  req.close()
  return data
 
f = open('vacancies.csv', mode='w', encoding='utf8')
writer = csv.writer(f)

headers = ["name", "area", "salary_from", "salary_to", "employer_name", "published_at", "schedule", "responsibility", "requirement"]
writer.writerow(headers)

for page in range(0, 20):
  jsObj = json.loads(getPage(page))

  for item in jsObj['items']:
    obj_to_write = []

    obj_to_write.append(item['name'])

    obj_to_write.append(item['area']['name'])

    if not item['salary'] is None:
      obj_to_write.append(item['salary']['from'])
      obj_to_write.append(item['salary']['to'])
    else:
      obj_to_write.append(None)
      obj_to_write.append(None)

    obj_to_write.append(item['employer']['name'])
    
    obj_to_write.append(item['published_at'])

    obj_to_write.append(item['schedule']['name'])

    obj_to_write.append(item['snippet']['responsibility'])
    obj_to_write.append(item['snippet']['requirement'])

    writer.writerow(obj_to_write)

  if (jsObj['pages'] - page) <= 1:
    break
  time.sleep(0.25)


f.close()
print('Старницы поиска собраны')

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('vacancies.csv')
print(df)

In [ ]:
# сгруппировать вакансии по названиям

gourps = [list(), list(), list(), list(), list(), list(), list()]

for i in range(len(df)) :
  name = df.iloc[i, 0].lower()
  if "developer" in name or "разработчик" in name or "программист" in name :
    gourps[0].append(df.iloc[i])
  elif "водитель" in name or "курьер" in name :
    gourps[1].append(df.iloc[i])
  elif "врач" in name:
    gourps[2].append(df.iloc[i])
  elif "менеджер" in name or "администратор" in name :
    gourps[3].append(df.iloc[i])
  elif "оператор" in name:
    gourps[4].append(df.iloc[i])
  elif "инженер" in name:
    gourps[5].append(df.iloc[i])
  else:
    gourps[6].append(df.iloc[i])

gourps_df[0]

In [ ]:
gourps_df = []

for i in range(7):
  gourps_df.append(pd.DataFrame(gourps[i]))

In [ ]:
# заполнить пропуски по зарплате средним значением


for i in range(len(gourps_df)):
  gourps_df[i]['salary_from'] = gourps_df[i]['salary_from'].fillna(gourps_df[i]['salary_from'].mean())
  gourps_df[i]['salary_to'] = gourps_df[i]['salary_to'].fillna(gourps_df[i]['salary_to'].mean())

df = df.merge(gourps_df[0], how='right')
df = df.merge(gourps_df[1], how='right')
df = df.merge(gourps_df[2], how='right')
df = df.merge(gourps_df[3], how='right')
df = df.merge(gourps_df[4], how='right')
df = df.merge(gourps_df[5], how='right')
df = df.merge(gourps_df[6], how='right')

df

In [ ]:
# добавить новый признак в датасет на основе признака “дата размещения вакансии” - количество дней с момента размещения;
from datetime import datetime, date


day_count = []
today = datetime.today()

for i in range(len(df)):
  published_at = datetime.strptime(df.iloc[i]['published_at'].split("T")[0], '%Y-%m-%d')
  day_count.append((today - published_at).days)

df['days_count'] = day_count

df

In [ ]:
#заполнить пропуски в признаке “требуемый опыт работы” по принципу: если не указан, то опыт не требуется;

for i in range(len(df)):
  if 'nan' in str(df.iloc[i]['requirement']):
    df.iloc[i, 8]  = "Не требуется"

df